In [1]:
import pandas as pd

In [6]:
import csv, sqlite3
%load_ext sql

In [7]:
con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

In [8]:
%sql sqlite:///FinalDB.db

In [9]:
df1 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01')
df2 = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
df3 = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")

In [10]:
df1.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")
df2.to_sql("CHICAGO_PUBLIC_SCHOOLS", con, if_exists="replace", index=False, method="multi")
df3.to_sql("CHICAGO_CRIME_DATA", con, if_exists="replace", index=False, method="multi")

533

## Problem 1 
Total number of crimes recorded in the CRIME table

In [12]:
cur.execute("SELECT COUNT(*) AS Total_Crimes FROM CHICAGO_CRIME_DATA;")
print(cur.fetchone()[0])

533


## Problem 2
List community area names and numbers with per capita income less than 11000

In [13]:
cur.execute("SELECT Community_Area_Name, Community_Area_Number FROM CENSUS_DATA WHERE Per_Capita_Income < 11000;")
print(cur.fetchall())

[('West Garfield Park', 26.0), ('South Lawndale', 30.0), ('Fuller Park', 37.0), ('Riverdale', 54.0)]


## Problem 3
List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [16]:
cur.execute("SELECT Case_Number FROM CHICAGO_CRIME_DATA WHERE Description LIKE '%MINOR%';")
print(cur.fetchall())

[('HL266884',), ('HK238408',)]


## Problem 4
List all kidnapping crimes involving a child?¶

In [19]:
cur.execute("SELECT * FROM CHICAGO_CRIME_DATA WHERE Description LIKE '%KIDNAPPING%' AND Description LIKE '%CHILD%';")
print(cur.fetchall())

[]


## Problem 5
List the kind of crimes that were recorded at schools. (No repetitions)

In [21]:
cur.execute("SELECT DISTINCT Primary_Type FROM CHICAGO_CRIME_DATA WHERE Location_Description LIKE '%SCHOOL%';")
print(cur.fetchall())

[('BATTERY',), ('CRIMINAL DAMAGE',), ('NARCOTICS',), ('ASSAULT',), ('CRIMINAL TRESPASS',), ('PUBLIC PEACE VIOLATION',)]


## Problem 6
List the type of schools along with the average safety score for each type.

In [28]:
df2.columns

Index(['School_ID', 'NAME_OF_SCHOOL', 'Elementary, Middle, or High School',
       'Street_Address', 'City', 'State', 'ZIP_Code', 'Phone_Number', 'Link',
       'Network_Manager', 'Collaborative_Name',
       'Adequate_Yearly_Progress_Made_', 'Track_Schedule',
       'CPS_Performance_Policy_Status', 'CPS_Performance_Policy_Level',
       'HEALTHY_SCHOOL_CERTIFIED', 'Safety_Icon', 'SAFETY_SCORE',
       'Family_Involvement_Icon', 'Family_Involvement_Score',
       'Environment_Icon', 'Environment_Score', 'Instruction_Icon',
       'Instruction_Score', 'Leaders_Icon', 'Leaders_Score', 'Teachers_Icon',
       'Teachers_Score', 'Parent_Engagement_Icon', 'Parent_Engagement_Score',
       'Parent_Environment_Icon', 'Parent_Environment_Score',
       'AVERAGE_STUDENT_ATTENDANCE', 'Rate_of_Misconducts__per_100_students_',
       'Average_Teacher_Attendance',
       'Individualized_Education_Program_Compliance_Rate', 'Pk_2_Literacy__',
       'Pk_2_Math__', 'Gr3_5_Grade_Level_Math__', 'Gr3_5_Gr

In [31]:
cur.execute("SELECT `Elementary, Middle, or High School` AS `Type of school`, AVG(SAFETY_SCORE) AS Avg_Safety_Score FROM CHICAGO_PUBLIC_SCHOOLS GROUP BY `Elementary, Middle, or High School`;")
print(cur.fetchall())

[('ES', 49.52038369304557), ('HS', 49.62352941176471), ('MS', 48.0)]


## Problem 7
List 5 community areas with highest % of households below poverty line¶

In [33]:
cur.execute("SELECT Community_Area_Name FROM CENSUS_DATA ORDER BY Percent_Households_Below_Poverty DESC LIMIT 5;")
print(cur.fetchall())

[('Riverdale',), ('Fuller Park',), ('Englewood',), ('North Lawndale',), ('East Garfield Park',)]


## Problem 8
Which community area is most crime prone? Display the coumminty area number only

In [34]:
cur.execute("SELECT Community_Area_Number FROM CHICAGO_CRIME_DATA GROUP BY Community_Area_Number ORDER BY COUNT(*) DESC LIMIT 1;")
print(cur.fetchone()[0])

25.0


## Problem 9
Use a sub-query to find the name of the community area with highest hardship index

In [35]:
cur.execute("SELECT Community_Area_Name FROM CENSUS_DATA WHERE Hardship_Index = (SELECT MAX(Hardship_Index) FROM CENSUS_DATA);")
print(cur.fetchone()[0])

Riverdale


## Problem 10
Use a sub-query to determine the Community Area Name with most number of crimes?¶

In [47]:
df3.columns

Index(['ID', 'CASE_NUMBER', 'DATE', 'BLOCK', 'IUCR', 'PRIMARY_TYPE',
       'DESCRIPTION', 'LOCATION_DESCRIPTION', 'ARREST', 'DOMESTIC', 'BEAT',
       'DISTRICT', 'WARD', 'COMMUNITY_AREA_NUMBER', 'FBICODE', 'X_COORDINATE',
       'Y_COORDINATE', 'YEAR', 'LATITUDE', 'LONGITUDE', 'LOCATION'],
      dtype='object')

In [49]:
df1.columns

Index(['COMMUNITY_AREA_NUMBER', 'COMMUNITY_AREA_NAME',
       'PERCENT_OF_HOUSING_CROWDED', 'PERCENT_HOUSEHOLDS_BELOW_POVERTY',
       'PERCENT_AGED_16__UNEMPLOYED',
       'PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA',
       'PERCENT_AGED_UNDER_18_OR_OVER_64', 'PER_CAPITA_INCOME',
       'HARDSHIP_INDEX'],
      dtype='object')

In [51]:
cur.execute("SELECT COMMUNITY_AREA_NUMBER FROM (SELECT COMMUNITY_AREA_NUMBER, COUNT(*) AS Crime_Count FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1);")
print(cur.fetchone()[0])

25.0


In [54]:
cur.execute("SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE COMMUNITY_AREA_NUMBER = 25.0")
print(cur.fetchone()[0])

Austin
